In [ ]:
# LAMBDA PYTHON IMPORT
import os
import sys
sys.path.insert(0, os.path.abspath('./code/'))

# PYSPARK IMPORT
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

# CUSTOM IMPORT
import entryPoint

In [ ]:
# IMPORT DATASET
path = "./data/subventions-accordees-et-refusees.csv"
df = spark.read.format('csv').options(header=True, inferSchema=True, sep =';').load(path)

newColumns = ['Num', 'Annee', 'Collectivite', 'Benificiaire', 'Siret', 'Objet', 'Montant', 'Direction', 'Nature', 'Secteur']
df = df.toDF(*newColumns)

In [ ]:
# Récupération des secteurs

secteurs = entryPoint.getAllSecteur(df)

for secteur in secteurs:
    print(secteur)

In [ ]:
# Récupérer uniquement un `Secteur` spécifique

entryPoint.selectSecteur(df, 'Vie et animation locale').take(10)